spreadsheet.ipynb

## Specifiche da implementare in Erlang
 Un worksheet deve essere una “matrice” di NxM celle ;
 Una cella può contenere qualsiasi tipo di dato primitivo o  il valore 'undef' . 
 Il modulo del worksheet condiviso deve prevedere alcune funzioni di interfaccia sintatticamente definite 
 1) new(name) -> spreadsheet | {error,reason} % Crea un nuovo foglio di nome “name” di dimensioni NxM di K tab (fogli)
  assegna il processo creatore come proprietario del foglio. I parametri N, M, K sono default nel modulo
 2) new(name, N, M, K) -> spreadsheet | {error,reason} % Crea un nuovo foglio di nome “name” di K fogli in cui ogni tab ha dimensioni NxM • assegna il processo creatore come proprietario del foglio

In [ ]:
-module(spreadsheet).
-export([new/1, new/4]).

-record(spreadsheet, {
    name,
    tabs = [],         % Lista di fogli (ogni foglio è una matrice NxM)
    owner = undefined  % Il proprietario del foglio
}).

% Funzione per creare un nuovo foglio con dimensioni default
new(Name) ->
    N = 10,  % Default N (numero di righe)
    M = 10,  % Default M (numero di colonne)
    K = 1,   % Default K (numero di tab)
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio con dimensioni specificate
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    % Verifica se il nome del foglio esiste già
    case whereis(Name) of
        undefined ->
            % Creazione del processo proprietario
            Owner = self(),
            % Creazione dei tab (K fogli, ciascuno una matrice NxM)
            Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
            % Creazione del record spreadsheet
            Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
            % Registrazione del processo con il nome del foglio
            register(Name, self()),
            % Memorizzazione dello stato
            loop(Spreadsheet);
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Loop principale del processo, dove si gestiscono i messaggi
loop(Spreadsheet) ->
    receive
        % Gestire messaggi per operazioni sul foglio di calcolo
        stop -> 
            ok; % Fermare il processo
        _Other ->
            loop(Spreadsheet)
    end.


## Aggiunta di specifiche di condivisione con politiche di accesso ai dati
share(spreadsheet, AccessPolicies) -> bool. Il proprietario del foglio può condidivere il foglio in Lettura o Scrittura con altri processi secondo AccessPolicies, che è una lista di {Proc,AP} dove:
 • Proc è un Pid/reg_name;
  • AP = read | write.
Le policy di accesso ad un foglio possono cambiare in qualsiasi momento.


In [ ]:
-module(spreadsheet).
-export([new/1, new/4, share/2]).

% Record aggiornato con access_policies
-record(spreadsheet, {
    name,
    tabs = [],
    owner = undefined, 
    access_policies = [] % Lista di {Proc, AP} per gestire le policy di accesso
}).

% Funzione per creare un nuovo foglio con dimensioni default
new(Name) ->
    N = 10,
    M = 10,
    K = 1,
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio con dimensioni specificate
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    case whereis(Name) of
        undefined ->
            Owner = self(),
            Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
            Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
            register(Name, self()),
            loop(Spreadsheet);
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},
            receive
                {share_result, Result} -> Result
            end
    end.

% Loop principale del processo, dove si gestiscono i messaggi
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, AccessPolicies} ->
            if
                From =:= Owner ->
                    % Aggiornare le politiche di accesso
                    NewState = State#spreadsheet{access_policies = AccessPolicies},
                    From ! {share_result, true},
                    loop(NewState);
                true ->
                    % Se non è il proprietario, ritorna un errore
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;
        stop -> 
            ok; % Fermare il processo
        _Other ->
            loop(State)
    end.


## implementazioni ulteriori sulla gestione delle politiche di accesso ai dati 
2) Deve esserci un meccanismo per salvare le policy di accesso su disco e ricaricarle al riavvio del sistema (persistenza delle policy).
3) Una funzione deve permettere di rimuovere o modificare le policy di accesso esistenti


In [ ]:
% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.

% Funzione per salvare le politiche di accesso su disco
save_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {save_policies, self()},
            receive
                {save_policies_result, Result} -> Result
            end
    end.

% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.


**PROBLEMA**:se il pid del processo Spreadsheet coincide con self() per far sì che  coincida con il processo che lo ha generato (implementazione del requisito di proprietà), la shell resta in ascolto di messaggi senza fornire prompt.

**SOLUZIONE**:modello basato su processi separati, dove il processo che gestisce lo stato del foglio di calcolo (Spreadsheet) viene creato come un processo figlio separato dal processo proprietario.Proprietario: Il processo che crea e controlla il foglio di calcolo.
Processo separato per Spreadsheet: Il processo Spreadsheet deve essere separato dal proprietario usando la funzione spawn/1 o spawn/3 per creare un processo separato che gestisce il ciclo loop/1 del foglio di calcolo

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, read_cell/3, write_cell/4]).

-record(spreadsheet, {
    name,
    tabs = [],
    owner = undefined,
    access_policies = []
}).

% Funzione per creare un nuovo foglio con dimensioni default (separato dal proprietario)
new(Name) ->
    N = 10,
    M = 10,
    K = 1,
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio di calcolo con un processo separato
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    case whereis(Name) of
        undefined ->
            Owner = self(),  % Il proprietario è il processo che chiama new/4
            Pid = spawn(spreadsheet, loop, [Name, Owner, N, M, K]),  % Crea il processo separato
            register(Name, Pid),  % Registra il nome del processo per facilitarne l'accesso
            {ok, Pid};  % Restituisce il Pid del processo creato
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).
    
% Loop che gestisce lo Stato
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    receive
        

        stop ->
            ok;  % Termina il processo
        _Other ->
            loop(State)  % Continua a ricevere messaggi
    end.



Il processo Spreadsheet è ora separato dal processo chiamante, e si possono inviare messaggi da shell o da altri processi.
Il processo Spreadsheet rimane in esecuzione in background con uno Stato persistente,  ricevendo messaggi fino a quando non viene fermato esplicitamente con stop.
Il processo chiamante che ha creato il foglio è salvato come proprietario, e posso implementare controlli di accesso basati su questo.

# Aggiunta di funzioni per la persistenza delle policy di accesso

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, share/2, remove_policy/2, save_policies/1, load_policies/1]).

%%.............

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},
            receive
                {share_result, Result} -> Result
            end
    end.

% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.
% Funzione per salvare le politiche di accesso su disco    
{save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);


% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).

% Loop 
loop(State = #spreadsheet{owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, AccessPolicies} ->
            if
                From =:= Owner ->
                    NewState = State#spreadsheet{access_policies = AccessPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;

        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->
                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);
                true ->
                    From ! {remove_policy_result, {error, not_owner}},
                    loop(State)
            end;

       

        

        {save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);

        {load_policies, From} ->
            % Codice per caricare le politiche da disco (implementazione seguente)
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [read]),
            {ok, Policies} = io:fread(File, "~p"),
            file:close(File),
            NewState = State#spreadsheet{access_policies = Policies},
            From ! {load_policies_result, ok},
            loop(NewState);

        stop -> ok;
        _Other -> loop(State)
    end.




### Aggiunta di funzioni per la persistenza delle policy di accesso

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, share/2, remove_policy/2, save_policies/1, load_policies/1]).

%%.............

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},
            receive
                {share_result, Result} -> Result
            end
    end.

% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.
% Funzione per salvare le politiche di accesso su disco    
{save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);


% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).

% Loop 
loop(State = #spreadsheet{owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, AccessPolicies} ->
            if
                From =:= Owner ->
                    NewState = State#spreadsheet{access_policies = AccessPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;

        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->
                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);
                true ->
                    From ! {remove_policy_result, {error, not_owner}},
                    loop(State)
            end;

       

        

        {save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);

        {load_policies, From} ->
            % Codice per caricare le politiche da disco (implementazione seguente)
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [read]),
            {ok, Policies} = io:fread(File, "~p"),
            file:close(File),
            NewState = State#spreadsheet{access_policies = Policies},
            From ! {load_policies_result, ok},
            loop(NewState);

        stop -> ok;
        _Other -> loop(State)
    end.




### Timeout per la receive
```css

after 5000 ->
    {error, timeout}
end

```

# Persistenza del foglio di calcolo sotto forma di CSV
Soddisfatti i requisiti iniziali con new/1,new/4,share/2 per la creazione e la gestione delle politiche di accesso il progetto deve soddisfare ulteriori requisiti:
1) garantire la persistenza del foglio di calcolo con salvataggio dei record #spreadsheet sotto forma  di file CSV tenendo presente che le funzioni di interfaccia richieste devono essere 
to_cvs(filename, spreadsheet) - > ok | {error, reason} 
from_cvs(filename) - > speadsheet | {error, reason}.
In più bisogna tener presente che il foglio di calcolo accetta valori di ogni tipo più undef

In [ ]:
% Funzione per salvare il foglio di calcolo in un file CSV, accetta il registered name  spreadsheet
to_csv(Filename, SpreadsheetName) ->
    % Recupera il PID associato al nome registrato
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            % Invia un messaggio al processo per ottenere lo stato del foglio di calcolo
            Pid ! {get_spreadsheet, self()},
            receive
                {spreadsheet_state, #spreadsheet{name = Name, tabs = Tabs}} ->
                    case file:open(Filename, [write]) of
                        {ok, File} ->
                            % Scrivi il nome del foglio
                            io:format(File, "Spreadsheet Name: ~s~n", [Name]),
                            % Salva ogni tab come riga CSV
                            lists:foreach(fun(Tab) -> save_tab_to_csv(File, Tab) end, Tabs),
                            file:close(File),
                            ok;
                        {error, Reason} ->
                            {error, Reason}
                    end
            after 5000 ->
                {error, timeout}
            end
    end.

% Funzione per salvare ogni tab come riga CSV
save_tab_to_csv(File, Tab) ->
    lists:foreach(fun(Row) ->
        Line = lists:map(fun(Cell) -> format_cell(Cell) end, Row),
        io:format(File, "~s~n", [string:join(Line, ",")])
    end, Tab).

% Formattazione delle celle per CSV
format_cell(undef) -> "undef";
format_cell(Cell) -> io_lib:format("~p", [Cell]).


To implement the functions for reading and writing values in the spreadsheet cells (get/5 and set/6) and their versions with timeouts, we can modify the spreadsheet process to handle these operations.
get/5: Retrieve the value at position (i, j) in tab k of the spreadsheet.
set/6: Write a value to the position (i, j) in tab k of the spreadsheet.

Both operations should be able to:
Access the specific tab.
Index into the matrix representing the tab to read or write values.
Support a version with timeout (get/6 and set/7).
We will implement these functions by interacting with the spreadsheet process and handling messages like {get, From, Tab, I, J} and {set, From, Tab, I, J, Val} in the spreadsheet's loop.

In [ ]:
% La funzione get/5 spedisce un messaggio al processo spreadsheet richiedendo il valore di una specifica cella del foglio di calcolo
get(Name, Tab, I, J) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {get, self(), Tab, I, J},
    receive
        {cell_value, Value} -> Value
    after 5000 -> % Default timeout of 5 seconds
        timeout
    end.
% La funzione get/6 esegue lo stesso codice impostando un valore specifico per il timeout
get(Name, Tab, I, J, Timeout) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {get, self(), Tab, I, J},
    receive
        {cell_value, Value} -> Value
    after Timeout -> 
        timeout
    end.


%La funzione set/6 spedisce un messaggio al processo spreadsheet per aggiornare una cella specifica con un nuovo valore
set(Name, Tab, I, J, Val) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {set, self(), Tab, I, J, Val},
    receive
        {set_result, Result} -> Result
    after 5000 -> % Default timeout of 5 seconds
        timeout
    end.

% set/7 specifica ulteriormente un valore di timeout desiderato
set(Name, Tab, I, J, Val, Timeout) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {set, self(), Tab, I, J, Val},
    receive
        {set_result, Result} -> Result
    after Timeout -> 
        timeout
    end.


# funzione loop/1 aggiornata,  con aggiunta delle funzioni di lettura e scrittura

In [ ]:
% loop che gestisce lo State del foglio di calcolo e le operazioni sui dati
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    io:format("Spreadsheet State loop started. Waiting for messages.~n"),
    receive
 
        % Handle get request 
        %la Tab(la matrice) restituita da lists:nth(Tab, Tabs) è assegnata a TabMatrix
        {get, From, Tab, I, J} ->
            io:format("Received get request for Tab: ~p, Row: ~p, Col: ~p~n", [Tab, I, J]),
            if
                Tab > length(Tabs) orelse Tab < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [Tab]),
                    From ! {cell_value, undef};
                true ->
                    TabMatrix = lists:nth(Tab, Tabs),
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                            From ! {cell_value, undef}; %If the indices are out of bounds, we return undef instead of trying to access an invalid position.
                        true ->
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                    From ! {cell_value, undef};
                                true ->
                                    Value = lists:nth(J, Row),
                                    
                                    From ! {cell_value, Value}
                            end
                    end
            end,
            loop(State);

% Handle set request
        {set, From, Tab, I, J, Val} ->
            io:format("Received set request for Tab: ~p, Row: ~p, Col: ~p, Val: ~p~n", [Tab, I, J, Val]),
            if
                Tab > length(Tabs) orelse Tab < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [Tab]),
                    From ! {set_result, false};
                true ->
                    TabMatrix = lists:nth(Tab, Tabs),
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                            From ! {set_result, false};
                        true ->
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                    From ! {set_result, false};
                                true ->
                                    io:format("Setting value ~p at Tab: ~p, Row: ~p, Col: ~p~n", [Val, Tab, I, J]),
                                    NewRow = replace_nth(J, Val, Row),
                                    NewTabMatrix = replace_nth(I, NewRow, TabMatrix),
                                    NewTabs = replace_nth(Tab, NewTabMatrix, Tabs),
                                    NewState = State#spreadsheet{tabs = NewTabs},
                                    From ! {set_result, true},
                                    loop(NewState)
                            end
                    end
            end;

       



        {get_spreadsheetPid, From} ->
            
                io:format("Received request for spreadsheet state from: ~p~n", [From]),
                io:format("State to be returned: ~p~n", [State]),  % Stampa il valore di `State`
                % Invia il record #spreadsheet{} indietro
                From ! {spreadsheet_state, State},
                io:format("Sent spreadsheet state to: ~p~n", [From]),
                loop(State);  % Continua ad ascoltare messaggi

        {share, From, NewPolicies} ->
            if
                From =:= Owner ->
                    % Aggiornare le politiche di accesso
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    % Se non è il proprietario, ritorna un errore
                    From ! {share_result,{error, not_owner}},
                    loop(State)
            end;
        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->
                    
                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);
          
                true ->
                    From ! {error, not_owner},
                    loop(State)
            end;
        % Gestire messaggi per operazioni sul foglio di calcolo
        stop -> 

            ok; % esce dal loop , ma attenzione non arresta Spreadsheet !!!
        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.

To do
info(name) -> Spreadsheet_info Le informazioni devono contenere almeno: 
• Numerodi celle x tab 
• I permessi di lettura e scrittura

testare le politiche di accesso e inserire il controllo di accesso su get e set
Controllare i valori ritornati dalle funzioni delle specifiche (bool, ok,  ..)

